In [1]:
#Kaggle

In [2]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
#from KaggleWord2VecUtility import KaggleWord2VecUtility
import pandas as pd
import numpy as np
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [128]:
def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review,"html5lib").get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)

In [129]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences

In [130]:
import pandas as pd
import numpy as np
df = pd.read_excel("/home/varun/Downloads/SentimentAnalysisValidation_Nov212017(1).xlsx")
df.rename(columns={'Overall Support CSAT': 'CSAT'}, inplace=True)
df.drop(columns={'Surveyid for internal use (e.g. RI link)','Interaction Type','Response Date',})
df = df[['CSAT','Survey language','Full Text']]
df_english = df[(df['Survey language']=='English') | (df['Survey language']=='English (United Kingdom)')]
df_english = df_english.drop(columns={'Survey language'})

#df_english['CSAT1'] = np.where((df_english["CSAT"]>=0 and df_english["CSAT"]<6),"low", ((row["df_english"]>5 and row["df_english"]<9),"average", "good"))
#.apply(lambda x : "low" if (df_english["CSAT"]>=0 and df_english["CSAT"]<6) else ("average" if (row["df_english"]>5 and row["df_english"]<9) else "good"))
#  np.where(df['age']>=50, 'yes', 'no') 
df_english['CSAT'] = df_english['CSAT'].apply(lambda x : "low" if (x>=0 and x<6) else ("average" if (x>5 and x<9) else "good"))
train = df_english[:int((.9)*len(df_english))].reset_index(drop=True)
test = df_english[int((.9)*len(df_english)):].reset_index(drop=True)

In [131]:
train.columns

Index(['CSAT', 'Full Text'], dtype='object')

In [132]:
print('The first review is:')
print(train["Full Text"][0])

The first review is:
Making excuses....Be honest on the date of delivery of money... Don't blame it to the bank. Fixed your system


In [133]:
clean_train_reviews = []
print("Cleaning and parsing the training set movie reviews...\n")
for i in range( 0, len(train["Full Text"])):
    clean_train_reviews.append(" ".join(review_to_wordlist(train["Full Text"][i], True)))



Cleaning and parsing the training set movie reviews...



In [134]:
clean_train_reviews[0]

'making excuses honest date delivery money blame bank fixed system'

In [135]:
# ****** Create a bag of words from the training set
#
print ("Creating the bag of words...\n")


# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.
vectorizer = CountVectorizer(analyzer = "word",   \
                         tokenizer = None,    \
                         preprocessor = None, \
                         stop_words = None,   \
                         max_features = 5000)


Creating the bag of words...



In [136]:
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

In [137]:
# Numpy arrays are easy to work with, so convert the result to an
# array
np.asarray(train_data_features)

array(<1762x3891 sparse matrix of type '<class 'numpy.int64'>'
	with 25763 stored elements in Compressed Sparse Row format>, dtype=object)

In [138]:
# ******* Train a random forest using the bag of words
#
print ("Training the random forest (this may take a while)...")


# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100)

# Fit the forest to the training set, using the bag of words as
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["CSAT"] )

Training the random forest (this may take a while)...


In [139]:
# Create an empty list and append the clean reviews one by one
clean_test_reviews = []
print ("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,len(test["Full Text"])):
    clean_test_reviews.append(" ".join(review_to_wordlist(test["Full Text"][i], True)))
    
len(clean_test_reviews)


Cleaning and parsing the test set movie reviews...



196

In [140]:
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
np.asarray(test_data_features)
test_data_features.shape


(196, 3891)

In [141]:
# Use the random forest to make sentiment label predictions
print ("Predicting test labels...\n")
result = forest.predict(test_data_features)

Predicting test labels...



In [142]:
from nltk.classify.util import accuracy
print(forest.score( test_data_features, test["CSAT"] ))

0.7244897959183674


In [143]:
# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"Full Text":test["Full Text"], "CSAT":test["CSAT"],"result":result} )

In [147]:
output


,Full Text,CSAT,result
0,I just had a few problems understanding the co...,average,good
1,very helpful and courteous.,good,good
2,The interest. Kindness. Patience.I try to rege...,good,low
3,Your employee went out of her way to get infor...,good,low
4,"Quick, polite & efficient.",good,good
5,i can not still use my account.please help i n...,low,low
6,.Not happy at all,low,good
7,fast service.no,average,good
8,Sara took ownership and progressed transaction...,good,low
9,I called because I do not have a Western Union...,average,low


In [152]:
# Use pandas to write the comma-separated output file
output.to_csv('Bag_of_Words_model.csv', encoding='utf8')#, quoting=3)
print("Wrote results to Bag_of_Words_model.csv")

Wrote results to Bag_of_Words_model.csv
